# Introduccion

Hola de nuevo, espero que no hayas tenido problemas con el modulo anteior de descarga :smile:, porque ahora solo se pone mas complicado :confounded:.

Ahora que ya tienes los archivos suplementarios como recordaras ya tienes los datos de expresion crudos de los samples del experimento en cuestion, sin embargo, como recordaras esto aun no es suficiente para integrar los datos pues tienen distinto identificadoress que nos los hacen comparables entre si:cry:.

Para solucionar este problema se tienen que hacer dos cosas: 
- En primera parsear los archivos suplementarios para poder recuperar: el valor de expresion, el probe_id, y la secuencia del probe
- Mapear las secuencias de los probes para saber a que locus tag pertenecen

# Parseo

Como viste en el entrenamiento a travez del uso de programacion basica en python o el uso de librerias como pandas es posible parsear facilemente archivos de diferentes formatos. Sin embargo, cuando no hay modulo de apoyo, para poder parserar los datos vas a tener que entender como es el formato de tu archivo para poder extraer lo que necesitas. 

Ahora bien, para hacer este vas a tener que buscar los manuales de los manufacturadores que explican el formato de sus archivos, sin embargo, para propositos de este ejercicio en teoria solo tienes que buscar el de uno, el de Agilent. Esto porque no te voy a perdir que parsees los archivos de Affymetrix y te voy a ayudar con los de Nimblegen :nerd_face:

In [22]:
# Empezamos con un pequeño ejemplo con solo un sample
import GEOparse as geo
import os
import ftplib
import gzip
import shutil

gsm = geo.get_GEO(geo="GSM6481068",destdir="Results")

# Ignora que tiene dos archivos suplementarios como es ejemplo pude revisarlos a mano xd
ftpath = gsm.metadata["supplementary_file"][0]

_,_,host,*path = ftpath.split("/")
path = os.path.join(*path)
ftp = ftplib.FTP(host)
ftp.login()

fileName = "GSM6481068_supp.gz"

with open(pathfile:=f"Results/{fileName}", "wb") as f:
  ftp.retrbinary(f"RETR {path}", f.write)

ftp.quit()

# Los archivos estan zipiados por lo que tienes que gunzipearlos

with gzip.open(pathfile, "rb") as gz:
            with open(GSM_pathunzip := pathfile.strip(".gz"), "wb") as out:
                shutil.copyfileobj(gz, out)

20-Jun-2025 12:30:20 DEBUG utils - Directory Results already exists. Skipping.
20-Jun-2025 12:30:20 INFO GEOparse - File already exist: using local version.
20-Jun-2025 12:30:20 INFO GEOparse - Parsing Results/GSM6481068.txt: 


In [23]:
# Podras pensar que solo con los samples basta, pero no en caso de nimblegen tambien se necesita el suplementario del GPL
gpl = geo.get_GEO(geo=gsm.metadata["platform_id"][0],destdir="Results")

ftpath = gpl.metadata["supplementary_file"][0]

_,_,host,*path = ftpath.split("/")
path = os.path.join(*path)
ftp = ftplib.FTP(host)
ftp.login()

fileName = f"{gsm.metadata['platform_id'][0]}_supp.gz"

with open(pathfile:=f"Results/{fileName}", "wb") as f:
  ftp.retrbinary(f"RETR {path}", f.write)

ftp.quit()

with gzip.open(pathfile, "rb") as gz:
            # Recuerda guardar el path de los archivos de salida para mayor comodidad
            with open(GPL_pathunzip := pathfile.strip(".gz"), "wb") as out:
                shutil.copyfileobj(gz, out)

20-Jun-2025 12:30:32 DEBUG utils - Directory Results already exists. Skipping.
20-Jun-2025 12:30:32 INFO GEOparse - File already exist: using local version.
20-Jun-2025 12:30:32 INFO GEOparse - Parsing Results/GPL10416.txt: 
20-Jun-2025 12:30:32 DEBUG GEOparse - PLATFORM: GPL10416


In [24]:
import pandas as pd 
# Dado que son archivos tabulares es bastante simple el seleccionar lo que queremos
gsm_supp = pd.read_csv(GSM_pathunzip,sep="\t",comment="#")
gpl_supp = pd.read_csv(GPL_pathunzip,sep="\t",comment="#")

In [25]:
# Veamos los archivos 
gsm_supp.head(2)

,IMAGE_ID,GENE_EXPR_OPTION,SEQ_ID,PROBE_ID,POSITION,X,Y,MATCH_INDEX,SEQ_URL,PM,MM
0,6610-423383-425-control_532,FORWARD,NC_000913,ECOLIP1,1,580,820,65084075,NaN,551.33,0.0
1,6610-423383-425-control_532,FORWARD,NC_000913,ECOLIP25,25,542,974,65084076,NaN,475.22,0.0


In [ ]:
gpl_supp.head(2)

,PROBE_DESIGN_ID,CONTAINER,DESIGN_NOTE,SELECTION_CRITERIA,SEQ_ID,PROBE_SEQUENCE,MISMATCH,MATCH_INDEX,FEATURE_ID,ROW_NUM,COL_NUM,PROBE_CLASS,PROBE_ID,POSITION,DESIGN_ID,X,Y
0,4215_0001_0001,REVERSE,NaN,0,NC_000913,TGCCGGTAACGTCTGAGCGTACAATCCGGCGCGTTTTACCGCATTA...,0,65949753,65949753,1,1,experimental,ECOLIP1708766,1708766,4215,1,1
1,4215_0023_0001,FORWARD,NaN,0,NC_000913,GAAGCGGCTCATTAACAGGAGTATAATGATGGATTTTTCTTTAACT...,0,65158015,65158015,1,23,experimental,ECOLIP1775192,1775192,4215,23,1


Como puedes notar cada uno tiene informacion que nos interesa, por un lado el suplementario del GSM tiene los valores crudos de la columna 'PM', y en el suplementario del GPL la secuencia asociada a ello, pero hay algo que los une, y ese es el 'PROBE_ID'

In [37]:
# Entonces sabiendo eso podemos concatenar solo las columnas que nos interesan
merge_sups = gsm_supp.loc[:,["PROBE_ID","PM"]].merge(gpl_supp.loc[:,["PROBE_ID","PROBE_SEQUENCE"]],on="PROBE_ID",how="inner")

Y listo tenemos nuestros datos de interes :smile:. Aunque bueno eso solo es para un tipo de archivo de nimblegen, los .pair, para el .xys cambian los nombre de las columnas pero eso te dejare descubrirlo :+1:

# Mapeo

Ahora se viene lo que todos esperabamos, el mapeo de las secuencas, para ello como ya les platique previmente usamos el programa de bowtie2 para ello, aunque para eso necesitamos el genoma de referenci, aunque tu no tienes problema porque basta con que elijas el de escherichia coli k12: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/

Donde solo necesitas el fna y gff

Entonces ya con el genoma de referencia en tu directorio solo tienes que correr bowtie 2 de la siguiente manera

In [ ]:
# Puedes correrlo en la linea de bash o en python como a mi me gusta
import os 
preffix = "GPL"
fnaFilePath = "/export/storage/users/aggonzal/Induccion/GCF_000005845.2_ASM584v2_genomic.fna.gz"
# Como en muchos alineadores necesitas indexar tu genoma para manejarlo de mejor manera
# Yo no lo corro porque ya lo tengo xd
command = f"bowtie2-build {fnaFilePath} {preffix} > {preffix + '.out'}"
os.system(command=command)

In [38]:
# Al alineador tienes que pasarle archivos fasta
with open(fasta_path:="Fasta_attemp_2.fasta","w") as file:
    for _,(probe,seq) in merge_sups[["PROBE_ID","PROBE_SEQUENCE"]].iterrows():
        file.write(f">{probe}\n{seq}\n")

In [ ]:
indice = ""# El indice generado con el genoma de referencia
out = "" # Nombre del output de salida
stats = "" # Archivo donde se guardan las estadisticas de corrida
command = f"bowtie2 -f {fasta_path} -x {indice} -S {out} 2 > {stats}"
os.system(command=command)

In [42]:
import rpy2#.robjects as robjects
import tempfile
import sys
from rpy2.rinterface_lib.embedded import RRuntimeError

bamFile = "/export/storage/users/aggonzal/Induccion/salida_2.bam"
gtf = "/export/storage/users/aggonzal/Induccion/GCF_000005845.2_ASM584v2_genomic.gtf"
feature = "gene"
outDir = "feature_test_2"

command = f'''
        library(Rsubread)
        featureCounts("{bamFile}",
        annot.ext="{gtf}",
        isGTFAnnotationFile=T,
        nthreads=10,
        GTF.featureType="{feature}",
        reportReads="CORE",
        reportReadsPath="{outDir}", 
        verbose=F, tmpDir="{outDir}")
        '''
#robjects.r(command)
# Todo esto es solo para el manejo de errores con la libreria
with tempfile.TemporaryFile(mode="w") as tempFile:
        __stdout__ = sys.stdout
        sys.stdout = tempFile
        try:
                rpy2.robjects.r(command)
        except RRuntimeError:
                print("Valio sorbete")
        
        sys.stdout = __stdout__



In [43]:
# Ya solo tienes que renotar los probes para que los valores esten asociados a sus locus tag correspondientes
import pandas as pd
features = pd.read_csv("feature_test_2/salida_2.bam.featureCounts",sep="\t",header=None)
features.columns = ["PROBE_ID","ASSIGN","VALUE","GENE"]
probe_feature = merge_sups.merge(features.loc[:,["PROBE_ID","GENE"]],on="PROBE_ID")


In [ ]:
# Para la matriz solo necesitas el valor y el nombre del gen
matriz = probe_feature.dropna()[["GENE","PM"]].drop_duplicates()
matriz.set_index("GENE",inplace=True)
# En este caso sabemos que todos nuestros valor pertenecen a un solo sample
matriz.rename(columns={"PM":"GSM6481068"},inplace=True)

In [75]:
# Y listo tenemos una columna de la matriz de expresion
matriz.head()

,GSM6481068
GENE,
b0001,312.33
b0001,473.33
b0001,739.33
b0001,454.78
b0001,162.22


# Actividad :horse: :fire: :fire:

Espero que los codigos que te di se pudieran entender :smiley_cat:, pero ahora es tiempo de ponerlo en practica como ya sabes, pero tranquilo si alguna cosa no funciona lo podemos revisar.

Para este modulo solo te voy a pedir lo siguiente:
Para aquellos gsms a los que les descargaste suplementarios, exclusivamente aquellos con manufacturadores de Nimblegen o Agilent; deberas realizar un parseo de los archivos suplementarios para la extraccion de minimo el valor crudo, la secuencia y el probe_id. 

Para posteriormente mapear las secuencias y recuperar el locus tag asociado a cada valor, ojo si algunas de las secuencias no mapean y se pierden esta bien. 

Finalmente una vez recuperados los locus tag construye una matriz de expresion donde las filas sean los locus tag y las columnas los gsms, para todos los gsms que pudiste recuperar.